In [1]:
## Big Picture Question: Can industry factors explain some of the excess returns of momentum portfolios?

## We will first do a time-series test of momentum portfolios after doing PDS lasso on our factors to choose the best ones

In [61]:
import statsmodels.formula.api as smf
from fama_data.FamaData import FamaData
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import warnings
import numpy as np
warnings.filterwarnings('ignore')

In [62]:
port = pd.read_csv('http://diether.org/prephd/10-port_mom_ew.csv',parse_dates=['caldt'])
port

,caldt,p0,p1,p2,p3,p4
0,1927-01-31,-2.708397,4.290747,0.776512,1.210773,0.508464
1,1927-02-28,5.418420,8.356758,4.758203,4.905527,5.422874
2,1927-03-31,-3.829973,-2.910900,-1.801945,0.259834,0.020529
3,1927-04-30,-0.747539,-0.438827,-0.216951,-0.172500,3.216826
4,1927-05-31,3.314240,5.759958,7.950440,8.014930,8.587076
...,...,...,...,...,...,...
1144,2022-05-31,-5.392614,-2.553349,0.502323,-0.183222,1.199527
1145,2022-06-30,-5.344633,-8.743887,-8.291228,-4.778448,-9.131728
1146,2022-07-29,11.776372,11.033183,9.284861,6.875124,8.193744
1147,2022-08-31,0.317134,-2.100848,-2.140245,-2.222316,0.123339


In [63]:
fac = pd.read_csv('http://diether.org/prephd/10-factor.csv',parse_dates=['caldt'])
fac.head()

,caldt,exmkt,smb,hml,umd,rf
0,1927-01-31,-0.06,-0.37,4.54,0.36,0.25
1,1927-02-28,4.18,0.04,2.94,-2.14,0.26
2,1927-03-31,0.13,-1.65,-2.61,3.61,0.30
3,1927-04-30,0.46,0.30,0.81,4.30,0.25
4,1927-05-31,5.44,1.53,4.73,3.00,0.30


In [64]:
port = port.merge(fac,on='caldt',how='inner')
port.head()
port=port.query('caldt >= "1963-07-31" and caldt <= "2023-02-28"')

In [65]:
names = ['exp0','exp1','exp2','exp3','exp4']
port[names] = port[[s[2:] for s in names]].sub(port['rf'],axis='index')
port.head()

,caldt,p0,p1,p2,p3,p4,exmkt,smb,hml,umd,rf,exp0,exp1,exp2,exp3,exp4
438,1963-07-31,-1.643322,-1.284810,-0.512732,-0.979859,-0.385561,-0.39,-0.45,-0.97,0.90,0.27,-1.913322,-1.554810,-0.782732,-1.249859,-0.655561
439,1963-08-30,2.761947,3.405911,4.273330,4.401903,6.501132,5.07,-0.98,1.80,1.01,0.25,2.511947,3.155911,4.023330,4.151903,6.251132
440,1963-09-30,-2.617577,-0.725381,-1.154537,-1.158321,-1.688062,-1.57,-0.33,0.13,0.19,0.27,-2.887577,-0.995381,-1.424537,-1.428321,-1.958062
441,1963-10-31,0.322188,0.710325,0.512692,1.902940,3.337489,2.53,-0.58,-0.10,3.12,0.29,0.032188,0.420325,0.222692,1.612940,3.047489
442,1963-11-29,-2.025639,-1.055293,-0.877067,-1.174566,-1.272383,-0.85,-1.17,1.75,-0.74,0.27,-2.295639,-1.325293,-1.147067,-1.444566,-1.542383


In [28]:
## Sanity check that we are doing this right as compared to the hw key
reg0 = smf.ols('exp0 ~ 1 + exmkt',data=port).fit()
reg0.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   exp0   R-squared:                       0.665
Model:                            OLS   Adj. R-squared:                  0.665
Method:                 Least Squares   F-statistic:                     1410.
Date:                Wed, 12 Apr 2023   Prob (F-statistic):          1.04e-170
Time:                        21:29:58   Log-Likelihood:                -2066.6
No. Observations:                 711   AIC:                             4137.
Df Residuals:                     709   BIC:                             4146.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.9259      0.167     -5.529      0.000      -1.255      -0.597
exmkt          1.3914      0.037     37.548      0.000       1.319       1.464
==============================================================================
Omnibus:                      233.008   Durbin-Watson:                   1.942
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1730.088
Skew:                           1.265   Prob(JB):                         0.00
Kurtosis:                      10.211   Cond. No.                         4.56
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [66]:
port['spread'] = port['p4'] - port['p0']

In [67]:
if_df = pd.read_csv('exported/lasso_industry_factors.csv')
ff_df = pd.read_csv('exported/alt_ff_factors.csv')
if_df

,AgricMinusSteel,AgricMinusChips,SodaMinusTxtls,HshldMinusClths,HshldMinusTxtls,HshldMinusAero,HshldMinusChips,HshldMinusBoxes,HshldMinusBanks,MedEqMinusChips,DrugsMinusHardw,DrugsMinusBanks,caldt
0,0.606461,0.584298,-0.162165,0.134815,-0.584478,0.766064,0.173176,-0.115244,0.310362,-0.056372,0.146697,0.420152,1963-07-31
1,-1.229077,-1.070887,0.144983,0.303882,0.321849,0.601337,-0.134279,0.163655,0.445749,0.254621,0.827600,1.035070,1963-08-31
2,-0.343146,0.259836,0.005897,0.552625,-0.028619,0.790125,0.420801,0.161331,0.375940,0.036411,-0.987600,-0.225803,1963-09-30
3,-0.271164,-1.422507,-0.698224,0.371898,-0.230750,0.284838,-0.902084,0.212463,1.181920,-0.068399,-0.953473,0.763498,1963-10-31
4,-0.204720,-0.694969,-0.401218,0.278619,-0.344856,-1.116269,-0.305456,0.028854,0.669985,-0.169772,-0.136064,0.241690,1963-11-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...
706,2.090395,1.240369,-0.405565,0.292223,-1.213691,-0.281527,-0.596291,-0.110595,-1.796609,-0.149154,0.733346,-0.270419,2022-05-31
707,1.153246,-0.176115,1.656091,1.868242,1.412375,-0.166773,1.462822,1.383838,2.087324,0.563895,1.437000,2.487985,2022-06-30
708,-1.994577,-1.532567,-0.547548,-2.123303,-0.985479,-0.981155,-2.691967,-1.453958,-1.637951,-1.776279,-1.364615,-1.616644,2022-07-31
709,0.862550,1.557684,1.101197,0.781935,1.645477,-0.039063,0.606936,1.632523,0.272284,0.137784,-0.035310,-0.413964,2022-08-31


In [68]:
df = pd.concat([ff_df.drop('caldt', axis = 1), if_df.drop('caldt', axis = 1)], axis=1)
df = df.drop('HshldMinusChips', axis = 1)
df.describe()

,exmt,share_issue_delta,shortterm_rev_delta,longterm_rev_delta,innov_delta,growth_delta,durable_delta,service_minus_goods,needs_minus_wants,accrual_delta,...,AgricMinusChips,SodaMinusTxtls,HshldMinusClths,HshldMinusTxtls,HshldMinusAero,HshldMinusBoxes,HshldMinusBanks,MedEqMinusChips,DrugsMinusHardw,DrugsMinusBanks
count,711.000000,711.000000,711.000000,711.000000,711.000000,711.000000,711.000000,711.000000,711.000000,711.000000,...,7.110000e+02,7.110000e+02,711.000000,7.110000e+02,7.110000e+02,7.110000e+02,7.110000e+02,7.110000e+02,7.110000e+02,7.110000e+02
mean,0.005433,-0.428439,-0.330380,-0.321969,0.011772,0.160225,0.044543,-4.241210,-6.706090,-0.327257,...,-7.495177e-18,-2.498392e-17,0.000000,4.996784e-18,-6.245980e-18,-8.744373e-18,-6.245980e-18,9.993569e-18,-4.996784e-18,1.249196e-18
std,0.044894,3.225668,5.529345,5.322832,5.171849,3.418934,4.277288,27.565854,31.480629,2.810405,...,1.000704e+00,1.000704e+00,1.000704,1.000704e+00,1.000704e+00,1.000704e+00,1.000704e+00,1.000704e+00,1.000704e+00,1.000704e+00
min,-0.232400,-12.380000,-37.920000,-33.380000,-26.100000,-14.485000,-13.520000,-135.230000,-174.720000,-13.730000,...,-3.077692e+00,-4.439050e+00,-7.669491,-8.151329e+00,-4.845767e+00,-6.185943e+00,-6.190361e+00,-3.877625e+00,-4.229608e+00,-4.735851e+00
25%,-0.019700,-2.395000,-2.695000,-3.075000,-2.785000,-1.800000,-2.255000,-14.585000,-12.400000,-1.865000,...,-6.656672e-01,-6.040516e-01,-0.554086,-4.980836e-01,-5.721134e-01,-5.916958e-01,-5.760044e-01,-5.675330e-01,-5.951459e-01,-5.866070e-01
50%,0.009100,-0.280000,-0.250000,-0.040000,-0.020000,0.135000,-0.220000,-0.680000,-0.570000,-0.280000,...,1.970531e-02,5.897122e-03,0.041536,-9.078950e-04,1.646257e-02,-6.008159e-03,5.739336e-03,1.063779e-02,3.456753e-02,4.770939e-02
75%,0.034000,1.540000,2.485000,2.680000,2.870000,2.137500,1.975000,10.810000,9.715000,1.245000,...,5.714338e-01,5.861452e-01,0.553597,4.669263e-01,6.059640e-01,5.715450e-01,5.853677e-01,6.274678e-01,5.651519e-01,5.976442e-01
max,0.161000,10.550000,25.390000,21.090000,24.370000,15.285000,33.860000,88.900000,75.750000,10.900000,...,4.167674e+00,5.086871e+00,2.917625,4.719816e+00,5.380278e+00,4.235579e+00,3.483509e+00,5.539772e+00,5.273783e+00,4.683844e+00


In [69]:
regressors = df.columns
df = pd.concat([port.reset_index(drop = True), df.reset_index(drop = True)], axis=1)
df

,caldt,p0,p1,p2,p3,p4,exmkt,smb,hml,umd,...,AgricMinusChips,SodaMinusTxtls,HshldMinusClths,HshldMinusTxtls,HshldMinusAero,HshldMinusBoxes,HshldMinusBanks,MedEqMinusChips,DrugsMinusHardw,DrugsMinusBanks
0,1963-07-31,-1.643322,-1.284810,-0.512732,-0.979859,-0.385561,-0.39,-0.45,-0.97,0.90,...,0.584298,-0.162165,0.134815,-0.584478,0.766064,-0.115244,0.310362,-0.056372,0.146697,0.420152
1,1963-08-30,2.761947,3.405911,4.273330,4.401903,6.501132,5.07,-0.98,1.80,1.01,...,-1.070887,0.144983,0.303882,0.321849,0.601337,0.163655,0.445749,0.254621,0.827600,1.035070
2,1963-09-30,-2.617577,-0.725381,-1.154537,-1.158321,-1.688062,-1.57,-0.33,0.13,0.19,...,0.259836,0.005897,0.552625,-0.028619,0.790125,0.161331,0.375940,0.036411,-0.987600,-0.225803
3,1963-10-31,0.322188,0.710325,0.512692,1.902940,3.337489,2.53,-0.58,-0.10,3.12,...,-1.422507,-0.698224,0.371898,-0.230750,0.284838,0.212463,1.181920,-0.068399,-0.953473,0.763498
4,1963-11-29,-2.025639,-1.055293,-0.877067,-1.174566,-1.272383,-0.85,-1.17,1.75,-0.74,...,-0.694969,-0.401218,0.278619,-0.344856,-1.116269,0.028854,0.669985,-0.169772,-0.136064,0.241690
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
706,2022-05-31,-5.392614,-2.553349,0.502323,-0.183222,1.199527,-0.34,-1.85,8.41,2.48,...,1.240369,-0.405565,0.292223,-1.213691,-0.281527,-0.110595,-1.796609,-0.149154,0.733346,-0.270419
707,2022-06-30,-5.344633,-8.743887,-8.291228,-4.778448,-9.131728,-8.43,2.09,-5.97,0.79,...,-0.176115,1.656091,1.868242,1.412375,-0.166773,1.383838,2.087324,0.563895,1.437000,2.487985
708,2022-07-29,11.776372,11.033183,9.284861,6.875124,8.193744,9.57,2.81,-4.10,-3.96,...,-1.532567,-0.547548,-2.123303,-0.985479,-0.981155,-1.453958,-1.637951,-1.776279,-1.364615,-1.616644
709,2022-08-31,0.317134,-2.100848,-2.140245,-2.222316,0.123339,-3.77,1.39,0.31,2.10,...,1.557684,1.101197,0.781935,1.645477,-0.039063,1.632523,0.272284,0.137784,-0.035310,-0.413964


In [18]:
names = ['exp0','exp1','exp2','exp3','exp4']
df[names] = df[[s[2:] for s in names]].sub(df['rf'],axis='index')
df.head()

,caldt,p0,p1,p2,p3,p4,exmkt,smb,hml,umd,...,AgricMinusChips,SodaMinusTxtls,HshldMinusClths,HshldMinusTxtls,HshldMinusAero,HshldMinusBoxes,HshldMinusBanks,MedEqMinusChips,DrugsMinusHardw,DrugsMinusBanks
0,1927-01-31,-2.708397,4.290747,0.776512,1.210773,0.508464,-0.06,-0.37,4.54,0.36,...,0.584298,-0.162165,0.134815,-0.584478,0.766064,-0.115244,0.310362,-0.056372,0.146697,0.420152
1,1927-02-28,5.418420,8.356758,4.758203,4.905527,5.422874,4.18,0.04,2.94,-2.14,...,-1.070887,0.144983,0.303882,0.321849,0.601337,0.163655,0.445749,0.254621,0.827600,1.035070
2,1927-03-31,-3.829973,-2.910900,-1.801945,0.259834,0.020529,0.13,-1.65,-2.61,3.61,...,0.259836,0.005897,0.552625,-0.028619,0.790125,0.161331,0.375940,0.036411,-0.987600,-0.225803
3,1927-04-30,-0.747539,-0.438827,-0.216951,-0.172500,3.216826,0.46,0.30,0.81,4.30,...,-1.422507,-0.698224,0.371898,-0.230750,0.284838,0.212463,1.181920,-0.068399,-0.953473,0.763498
4,1927-05-31,3.314240,5.759958,7.950440,8.014930,8.587076,5.44,1.53,4.73,3.00,...,-0.694969,-0.401218,0.278619,-0.344856,-1.116269,0.028854,0.669985,-0.169772,-0.136064,0.241690


In [33]:
df['spread']=df['exp4']-df['exp0']
df

,caldt,p0,p1,p2,p3,p4,exmkt,smb,hml,umd,...,AgricMinusChips,SodaMinusTxtls,HshldMinusClths,HshldMinusTxtls,HshldMinusAero,HshldMinusBoxes,HshldMinusBanks,MedEqMinusChips,DrugsMinusHardw,DrugsMinusBanks
0,1963-07-31,-1.643322,-1.284810,-0.512732,-0.979859,-0.385561,-0.39,-0.45,-0.97,0.90,...,0.584298,-0.162165,0.134815,-0.584478,0.766064,-0.115244,0.310362,-0.056372,0.146697,0.420152
1,1963-08-30,2.761947,3.405911,4.273330,4.401903,6.501132,5.07,-0.98,1.80,1.01,...,-1.070887,0.144983,0.303882,0.321849,0.601337,0.163655,0.445749,0.254621,0.827600,1.035070
2,1963-09-30,-2.617577,-0.725381,-1.154537,-1.158321,-1.688062,-1.57,-0.33,0.13,0.19,...,0.259836,0.005897,0.552625,-0.028619,0.790125,0.161331,0.375940,0.036411,-0.987600,-0.225803
3,1963-10-31,0.322188,0.710325,0.512692,1.902940,3.337489,2.53,-0.58,-0.10,3.12,...,-1.422507,-0.698224,0.371898,-0.230750,0.284838,0.212463,1.181920,-0.068399,-0.953473,0.763498
4,1963-11-29,-2.025639,-1.055293,-0.877067,-1.174566,-1.272383,-0.85,-1.17,1.75,-0.74,...,-0.694969,-0.401218,0.278619,-0.344856,-1.116269,0.028854,0.669985,-0.169772,-0.136064,0.241690
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
706,2022-05-31,-5.392614,-2.553349,0.502323,-0.183222,1.199527,-0.34,-1.85,8.41,2.48,...,1.240369,-0.405565,0.292223,-1.213691,-0.281527,-0.110595,-1.796609,-0.149154,0.733346,-0.270419
707,2022-06-30,-5.344633,-8.743887,-8.291228,-4.778448,-9.131728,-8.43,2.09,-5.97,0.79,...,-0.176115,1.656091,1.868242,1.412375,-0.166773,1.383838,2.087324,0.563895,1.437000,2.487985
708,2022-07-29,11.776372,11.033183,9.284861,6.875124,8.193744,9.57,2.81,-4.10,-3.96,...,-1.532567,-0.547548,-2.123303,-0.985479,-0.981155,-1.453958,-1.637951,-1.776279,-1.364615,-1.616644
709,2022-08-31,0.317134,-2.100848,-2.140245,-2.222316,0.123339,-3.77,1.39,0.31,2.10,...,1.557684,1.101197,0.781935,1.645477,-0.039063,1.632523,0.272284,0.137784,-0.035310,-0.413964


In [34]:
## Why are these two results different?
smf.ols('spread ~ 1 + ' + ' + '.join(regressors), data=df).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 spread   R-squared:                       0.277
Model:                            OLS   Adj. R-squared:                  0.250
Method:                 Least Squares   F-statistic:                     10.10
Date:                Wed, 12 Apr 2023   Prob (F-statistic):           8.29e-34
Time:                        21:32:18   Log-Likelihood:                -2071.1
No. Observations:                 711   AIC:                             4196.
Df Residuals:                     684   BIC:                             4319.
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                1.3294      0.188      7.073      0.000       0.960       1.698
exmt                     5.0877      5.728      0.888      0.375      -6.159      16.334
share_issue_delta        0.0201      0.071      0.284      0.776      -0.119       0.159
shortterm_rev_delta      0.1611      0.034      4.733      0.000       0.094       0.228
longterm_rev_delta      -0.1300      0.042     -3.115      0.002      -0.212      -0.048
innov_delta              0.1246      0.087      1.431      0.153      -0.046       0.296
growth_delta            -0.1888      0.109     -1.736      0.083      -0.402       0.025
durable_delta           -0.1892      0.045     -4.227      0.000      -0.277      -0.101
service_minus_goods      0.0091      0.007      1.249      0.212      -0.005       0.023
needs_minus_wants       -0.0033      0.006     -0.529      0.597      -0.016       0.009
accrual_delta           -0.2537      0.064     -3.939      0.000      -0.380      -0.127
cashflow_price_delta    -0.1250      0.091     -1.369      0.171      -0.304       0.054
market_beta_delta       -0.0922      0.061     -1.519      0.129      -0.211       0.027
dividend_yield_delta    -0.3162      0.051     -6.162      0.000      -0.417      -0.215
earnings_price_delta     0.2021      0.093      2.170      0.030       0.019       0.385
variance_delta          -0.0874      0.045     -1.928      0.054      -0.176       0.002
AgricMinusSteel          0.5162      0.275      1.874      0.061      -0.025       1.057
AgricMinusChips          0.1348      0.344      0.391      0.696      -0.541       0.811
SodaMinusTxtls           0.0623      0.273      0.228      0.820      -0.474       0.598
HshldMinusClths          0.2500      0.249      1.006      0.315      -0.238       0.738
HshldMinusTxtls          0.1060      0.325      0.326      0.744      -0.532       0.744
HshldMinusAero           0.2595      0.226      1.148      0.252      -0.184       0.703
HshldMinusBoxes          0.2166      0.215      1.008      0.314      -0.205       0.638
HshldMinusBanks         -0.0860      0.415     -0.207      0.836      -0.901       0.729
MedEqMinusChips         -0.0786      0.281     -0.280      0.780      -0.630       0.473
DrugsMinusHardw          0.3292      0.370      0.889      0.374      -0.398       1.056
DrugsMinusBanks          0.9371      0.439      2.134      0.033       0.075       1.799
==============================================================================
Omnibus:                      104.040   Durbin-Watson:                   1.976
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              746.970
Skew:                          -0.410   Prob(JB):                    6.27e-163
Kurtosis:                       7.954   Cond. No.                     1.11e+03
==================

In [36]:
smf.ols('p4 - p0 ~ 1 + ' + ' + '.join(regressors), data=df).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     p4   R-squared:                       0.828
Model:                            OLS   Adj. R-squared:                  0.822
Method:                 Least Squares   F-statistic:                     126.7
Date:                Wed, 12 Apr 2023   Prob (F-statistic):          2.13e-241
Time:                        21:35:11   Log-Likelihood:                -1680.2
No. Observations:                 711   AIC:                             3414.
Df Residuals:                     684   BIC:                             3538.
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                1.1978      0.108     11.044      0.000       0.985       1.411
exmt                    93.8040      3.306     28.378      0.000      87.314     100.294
share_issue_delta        0.1328      0.041      3.252      0.001       0.053       0.213
shortterm_rev_delta      0.0454      0.020      2.311      0.021       0.007       0.084
longterm_rev_delta      -0.1428      0.024     -5.929      0.000      -0.190      -0.096
innov_delta              0.0141      0.050      0.280      0.780      -0.085       0.113
growth_delta            -0.0827      0.063     -1.318      0.188      -0.206       0.041
durable_delta           -0.1185      0.026     -4.588      0.000      -0.169      -0.068
service_minus_goods      0.0116      0.004      2.764      0.006       0.003       0.020
needs_minus_wants       -0.0052      0.004     -1.435      0.152      -0.012       0.002
accrual_delta           -0.0633      0.037     -1.702      0.089      -0.136       0.010
cashflow_price_delta    -0.0205      0.053     -0.389      0.697      -0.124       0.083
market_beta_delta        0.0260      0.035      0.741      0.459      -0.043       0.095
dividend_yield_delta    -0.1234      0.030     -4.169      0.000      -0.182      -0.065
earnings_price_delta     0.1444      0.054      2.686      0.007       0.039       0.250
variance_delta           0.2075      0.026      7.934      0.000       0.156       0.259
AgricMinusSteel          0.3820      0.159      2.403      0.017       0.070       0.694
AgricMinusChips          0.0692      0.199      0.348      0.728      -0.321       0.459
SodaMinusTxtls           0.1396      0.158      0.886      0.376      -0.170       0.449
HshldMinusClths          0.0326      0.143      0.227      0.820      -0.249       0.314
HshldMinusTxtls         -0.6453      0.187     -3.443      0.001      -1.013      -0.277
HshldMinusAero           0.0559      0.130      0.429      0.668      -0.200       0.312
HshldMinusBoxes          0.0990      0.124      0.799      0.425      -0.144       0.342
HshldMinusBanks          0.2646      0.240      1.104      0.270      -0.206       0.735
MedEqMinusChips          0.2917      0.162      1.801      0.072      -0.026       0.610
DrugsMinusHardw         -0.0407      0.214     -0.191      0.849      -0.460       0.379
DrugsMinusBanks          0.4938      0.253      1.949      0.052      -0.004       0.991
==============================================================================
Omnibus:                       75.913   Durbin-Watson:                   1.682
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              515.812
Skew:                           0.123   Prob(JB):                    9.84e-113
Kurtosis:                       7.165   Cond. No.                     1.11e+03
==================

In [71]:
fd = FamaData('data/', end_date = '2022-09-30')
SMB_proxy = fd.get_size_data()['Hi 10'] - fd.get_size_data()['Lo 10']
HML_proxy = fd.get_book_to_market_data()['Hi 10'] - fd.get_book_to_market_data()['Lo 10']

# Fama-French 5-factor model components
RMW_proxy = fd.get_operating_profitability_data()['Hi 10'] - fd.get_operating_profitability_data()['Lo 10']
CMA_proxy = fd.get_investment_data()['Hi 10'] - fd.get_investment_data()['Lo 10']

# And momentum
MOM_proxy = fd.get_momentum_data()['Hi PRIOR'] - fd.get_momentum_data()['Lo PRIOR']

SMB_proxy.name = 'SMB'
HML_proxy.name = 'HML'
RMW_proxy.name = 'RMW'
CMA_proxy.name = 'CMA'
MOM_proxy.name = 'MOM'

In [72]:
fama_factors = pd.concat([SMB_proxy.reset_index(drop=True), HML_proxy.reset_index(drop=True), RMW_proxy.reset_index(drop=True), 
                        CMA_proxy.reset_index(drop=True), MOM_proxy.reset_index(drop=True)], axis = 1)

df = pd.concat([df.reset_index(drop=True), fama_factors.reset_index(drop=True)], axis = 1)
df.head(3)

,caldt,p0,p1,p2,p3,p4,exmkt,smb,hml,umd,...,HshldMinusBoxes,HshldMinusBanks,MedEqMinusChips,DrugsMinusHardw,DrugsMinusBanks,SMB,HML,RMW,CMA,MOM
0,1963-07-31,-1.643322,-1.284810,-0.512732,-0.979859,-0.385561,-0.39,-0.45,-0.97,0.90,...,-0.115244,0.310362,-0.056372,0.146697,0.420152,0.55,-2.66,2.42,3.07,2.98
1,1963-08-30,2.761947,3.405911,4.273330,4.401903,6.501132,5.07,-0.98,1.80,1.01,...,0.163655,0.445749,0.254621,0.827600,1.035070,2.67,1.67,0.54,2.32,3.23
2,1963-09-30,-2.617577,-0.725381,-1.154537,-1.158321,-1.688062,-1.57,-0.33,0.13,0.19,...,0.161331,0.375940,0.036411,-0.987600,-0.225803,-0.73,-1.77,2.05,-1.02,1.16


In [76]:
## We don't want to include MOM if we are basing this on the momentum portfolios right?

regressors=regressors.append(pd.Index(['SMB','HML','RMW','CMA']))
regressors

Index(['exmt', 'share_issue_delta', 'shortterm_rev_delta',
       'longterm_rev_delta', 'innov_delta', 'growth_delta', 'durable_delta',
       'service_minus_goods', 'needs_minus_wants', 'accrual_delta',
       'cashflow_price_delta', 'market_beta_delta', 'dividend_yield_delta',
       'earnings_price_delta', 'variance_delta', 'AgricMinusSteel',
       'AgricMinusChips', 'SodaMinusTxtls', 'HshldMinusClths',
       'HshldMinusTxtls', 'HshldMinusAero', 'HshldMinusBoxes',
       'HshldMinusBanks', 'MedEqMinusChips', 'DrugsMinusHardw',
       'DrugsMinusBanks', 'SMB', 'HML', 'RMW', 'CMA'],
      dtype='object')

In [77]:
## We get a less R2 especially on the spread strategy. So basically this is saying that the industry factors can't explain momentum. 
smf.ols('spread ~ 1 + ' + ' + '.join(regressors), data=df).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 spread   R-squared:                       0.328
Model:                            OLS   Adj. R-squared:                  0.298
Method:                 Least Squares   F-statistic:                     11.05
Date:                Wed, 12 Apr 2023   Prob (F-statistic):           1.78e-41
Time:                        21:47:35   Log-Likelihood:                -2045.4
No. Observations:                 711   AIC:                             4153.
Df Residuals:                     680   BIC:                             4294.
Df Model:                          30                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                1.1014      0.185      5.950      0.000       0.738       1.465
exmt                    15.1775      5.748      2.641      0.008       3.892      26.463
share_issue_delta        0.0097      0.074      0.131      0.896      -0.136       0.156
shortterm_rev_delta      0.1458      0.033      4.397      0.000       0.081       0.211
longterm_rev_delta      -0.0177      0.046     -0.381      0.703      -0.109       0.073
innov_delta              0.1930      0.086      2.233      0.026       0.023       0.363
growth_delta            -0.1572      0.106     -1.487      0.138      -0.365       0.050
durable_delta           -0.1629      0.044     -3.731      0.000      -0.249      -0.077
service_minus_goods      0.0040      0.007      0.569      0.570      -0.010       0.018
needs_minus_wants       -0.0004      0.006     -0.067      0.947      -0.012       0.012
accrual_delta           -0.2680      0.064     -4.188      0.000      -0.394      -0.142
cashflow_price_delta    -0.1056      0.090     -1.177      0.240      -0.282       0.071
market_beta_delta       -0.1466      0.059     -2.468      0.014      -0.263      -0.030
dividend_yield_delta    -0.3390      0.050     -6.762      0.000      -0.437      -0.241
earnings_price_delta     0.1467      0.093      1.585      0.113      -0.035       0.328
variance_delta          -0.2079      0.050     -4.169      0.000      -0.306      -0.110
AgricMinusSteel          0.2985      0.273      1.092      0.275      -0.238       0.835
AgricMinusChips          0.1501      0.339      0.443      0.658      -0.515       0.815
SodaMinusTxtls           0.0168      0.265      0.063      0.950      -0.503       0.536
HshldMinusClths          0.2558      0.243      1.053      0.293      -0.221       0.733
HshldMinusTxtls          0.4841      0.323      1.500      0.134      -0.150       1.118
HshldMinusAero           0.2966      0.222      1.333      0.183      -0.140       0.733
HshldMinusBoxes          0.1068      0.209      0.511      0.610      -0.303       0.517
HshldMinusBanks         -0.1751      0.406     -0.431      0.667      -0.973       0.623
MedEqMinusChips         -0.1417      0.275     -0.516      0.606      -0.681       0.398
DrugsMinusHardw          0.4738      0.362      1.308      0.191      -0.238       1.185
DrugsMinusBanks          0.6599      0.432      1.526      0.127      -0.189       1.509
SMB                     -0.3457      0.059     -5.819      0.000      -0.462      -0.229
HML                     -0.0650      0.070     -0.927      0.354      -0.203       0.073
RMW                     -0.0215      0.064     -0.338      0.736      -0.147       0.104
CMA                     -0.2170      0.069     -3.152      0.002      -0.352      -0.082
=========================================================

In [79]:
smf.ols('exp0 ~ 1 + ' + ' + '.join(regressors), data=df).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   exp0   R-squared:                       0.866
Model:                            OLS   Adj. R-squared:                  0.861
Method:                 Least Squares   F-statistic:                     147.1
Date:                Wed, 12 Apr 2023   Prob (F-statistic):          2.86e-274
Time:                        21:49:41   Log-Likelihood:                -1740.0
No. Observations:                 711   AIC:                             3542.
Df Residuals:                     680   BIC:                             3684.
Df Model:                          30                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               -0.5913      0.120     -4.908      0.000      -0.828      -0.355
exmt                    94.8292      3.741     25.351      0.000      87.485     102.174
share_issue_delta       -0.0166      0.048     -0.343      0.732      -0.112       0.079
shortterm_rev_delta     -0.1069      0.022     -4.954      0.000      -0.149      -0.065
longterm_rev_delta       0.0067      0.030      0.224      0.823      -0.052       0.066
innov_delta             -0.0911      0.056     -1.620      0.106      -0.202       0.019
growth_delta             0.1026      0.069      1.491      0.136      -0.032       0.238
durable_delta            0.0946      0.028      3.329      0.001       0.039       0.150
service_minus_goods     -0.0017      0.005     -0.377      0.706      -0.011       0.007
needs_minus_wants     9.623e-06      0.004      0.002      0.998      -0.008       0.008
accrual_delta            0.1758      0.042      4.221      0.000       0.094       0.258
cashflow_price_delta     0.1035      0.058      1.772      0.077      -0.011       0.218
market_beta_delta        0.0857      0.039      2.217      0.027       0.010       0.162
dividend_yield_delta     0.1933      0.033      5.923      0.000       0.129       0.257
earnings_price_delta    -0.0953      0.060     -1.583      0.114      -0.214       0.023
variance_delta           0.2108      0.032      6.493      0.000       0.147       0.275
AgricMinusSteel         -0.2242      0.178     -1.261      0.208      -0.573       0.125
AgricMinusChips         -0.0253      0.221     -0.115      0.909      -0.458       0.408
SodaMinusTxtls           0.0157      0.172      0.091      0.928      -0.323       0.354
HshldMinusClths         -0.2153      0.158     -1.362      0.173      -0.526       0.095
HshldMinusTxtls         -0.4539      0.210     -2.161      0.031      -0.866      -0.041
HshldMinusAero          -0.1320      0.145     -0.912      0.362      -0.416       0.152
HshldMinusBoxes         -0.1484      0.136     -1.091      0.276      -0.415       0.119
HshldMinusBanks          0.1855      0.264      0.702      0.483      -0.334       0.705
MedEqMinusChips          0.2353      0.179      1.316      0.189      -0.116       0.586
DrugsMinusHardw         -0.2580      0.236     -1.094      0.274      -0.721       0.205
DrugsMinusBanks         -0.4366      0.281     -1.552      0.121      -0.989       0.116
SMB                     -0.2447      0.039     -6.331      0.000      -0.321      -0.169
HML                      0.0374      0.046      0.820      0.412      -0.052       0.127
RMW                     -0.0465      0.041     -1.120      0.263      -0.128       0.035
CMA                      0.1944      0.045      4.337      0.000       0.106       0.282
=========================================================